# Bending energy

I have a hypothesis that the active nematics bend more in slower flowing channels, resulting in more free energy penalty. It would be great if I can do a simulation on a constrained system, to see how the orientation field is optimized. However, since the simulation method is not readily available, I'm going to test this hypothesis on experimental data first. The plan is:

- extract orientational field $p$ from images;
- compute bending energy based on $p$ and the boundaries;
- plot bending energy as a function of flow rate (fraction).

In this notebook, I'm going to develop the method for image analysis, based on the code I got from Claire (originally from A. N.-F.'s lab).

## 0 Packages

In [1]:
import os
from skimage import io
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams["font.family"] = "STIXGeneral"
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams["xtick.direction"] = "in"
matplotlib.rcParams["ytick.direction"] = "in"
plt.rcParams['xtick.major.size'] = 2  # Length of major ticks
plt.rcParams['ytick.major.size'] = 2  # Length of major ticks
plt.rcParams['xtick.minor.size'] = 1  # Length of minor ticks
plt.rcParams['ytick.minor.size'] = 1  # Length of minor ticks

## 1 Filament orientation